In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

X_train = pd.read_csv("../Data/Processed/X_train.csv")
X_test  = pd.read_csv("../Data/Processed/X_test.csv")
# squeeze  making them a series/1D
y_train = pd.read_csv("../Data/Processed/y_train.csv").squeeze()
y_test  = pd.read_csv("../Data/Processed/y_test.csv").squeeze()



X = X_train
y = y_train

numeric_cols = [
    "Administrative", "Informational","ProductRelated", "Administrative_Duration", "Informational_Duration", "ProductRelated_Duration","BounceRates","ExitRates","SpecialDay",
]

# Fit the scaler on training numerics only then replace those columns in X
scaler = StandardScaler()
X[numeric_cols] = pd.DataFrame(
    scaler.fit_transform(X[numeric_cols]),
    columns=numeric_cols,
    index=X.index
).astype(float)


# class weight balanced counters the class imbalance (giving buyer errors more weight)
logr = LogisticRegression(class_weight='balanced', max_iter=200, random_state=42)
logr.fit(X, y)

print("Intercept:", logr.intercept_)
print("Coefficients shape:", logr.coef_.shape)



Intercept: [-0.59808947]
Coefficients shape: (1, 58)
